In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import openpyxl as px
import streamlit as st
from unidecode import unidecode  # For removing accents

In [ ]:
con_b = sql.connect("data/budget.db")
c_b= con_b.cursor()
# Lecture du fichier Excel
df = pd.read_excel("budget.xlsx")
# Insertion des données dans la table Budget
df.to_sql("budget", con_b, if_exists="replace", index=False)
con_b.commit()
con_b.close()

: 

In [16]:
df

,code_activite,projet,code_resultat,item_code,montant,departement
0,101,P001,R001,I001,5000,as
1,102,P001,R002,I002,3000,has
2,103,P002,R001,I003,1500,as
3,104,P003,R003,I004,2500,has
4,105,P004,R004,I005,4500,ase


In [ ]:
wb=px.load_workbook(r"C:\Users\nmaxi\Downloads\Fichier momo-PFormation_Entrepreneur_MIMIN_Clean.xlsx") #charger le classeur brut

# Get data from worksheet 'DEMANDE DE PAIEMENT MOMO' and get data from 1st column and rows contains "N°"
ws = wb.worksheets[0]
# Initialize variables
min_row = None
max_row = None
min_col = 1
max_col = None

# Find the min_row containing "NOM ET PRENOM MOMO"
for row in ws.iter_rows(min_row=1, max_col=ws.max_column):
    for cell in row:
        if cell.value == "NOM ET PRENOMS MOMO":
            min_row = cell.row
            break
    if min_row:
        break

# Find the max_row in the first column that contains a digit
for row in ws.iter_rows(min_row=min_row, max_row=ws.max_row, min_col=1, max_col=1):
    for cell in row:
        if isinstance(cell.value, int):
            max_row = cell.row

# Find the last column of the min_row containing "NOM CONFORME"
if min_row is not None:
    for cell in ws[min_row]:
        if cell.value is not None and cell.value == "NOM CONFORME":
            max_col = cell.column

# Now you can use these values to specify the range
rows = ws.iter_rows(min_row=min_row, max_row=max_row, min_col=min_col, max_col=max_col)

data = []
for row in rows:
    data.append([cell.value for cell in row])

# Create a dataframe from the data
df = pd.DataFrame(data)
# Consider second line of datframe as column names
df.columns = df.iloc[0]
# delete column whose name doesn't contains list of strings
df=df[df.filter(regex="Nom|NOM|nom|Prénom|PRENOM|prenom|prénom|phone|Phone|PHONE|STATUT").columns]

# Drop 1st row
df = df.drop(0)

# Keep only last 4 columns
df = df.iloc[:, -4:]

# Rename first column to 'numero'
df.rename(columns={df.columns[0]:'numero'},inplace=True)

# subset the dataframe to keep only rows where statut is not active and NOM conforme is not 1 or len of phone number is not 8, 10, 11 or 13
df_nc = df[(df['STATUT']!='ACTIVE') | (df['NOM CONFORME']!=1) | (df['numero'].apply(lambda x: len(str(x)) not in [8,10,11,13]))]

# Add "229" to numbers where lenghts is 10 digits and start with 01, "22901" where lenghts is 8 digits,
# insert 01 beetwen 229 and the rest of the number where lenghts is 11
# keep as is it where len is 13 digits and raise erros for other cases
df['numero'] = df['numero'].apply(lambda x: '229'+x if len(str(x))==10 and str(x).startswith('01')
                                  else ('22901'+x if len(str(x))==8 else ('22901'+str(x)[3:] if len(str(x))==11 else 'error')))

# Convert only columns whone contains phone, or Phone or PHONE to numeric
df['numero']= df['numero'].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values for phone numbers
df = df.dropna(subset=['numero'])


# subset the dataframe to keep only rows where statut is active and NOM conforme is 1
df_o = df[(df['STATUT']=='ACTIVE') & (df['NOM CONFORME']==1)]

# rename column nom et prenom momo in nom
df_o.rename(columns={df.columns[1]:'nom'},inplace=True)

# Drop colums whose not contains nom or numero
df_o = df_o[df_o.filter(regex="nom|numero").columns]

# Create a connection to a sqlite database
con=sql.connect('data/AffiKoue.db')
# Create a cursor
c=con.cursor()
# fetching number values from sqlite table momo
num_exist=pd.read_sql_query("SELECT numero FROM momo",con)['numero'].tolist()
# create dataframe for only rows where numero doesn't exist in table data
df_i = df_o[~df_o['numero'].isin(num_exist)]
# create dataframe for only rows where numero exist in table data
df_e = df_o[df_o['numero'].isin(num_exist)]
# insert df_o rows into table data into affikoue db, if exist append the rows where the numero doesn't exist in table data
if not df_i.empty:
    df_o.to_sql('momo',con,if_exists='append',index=False)
    print(f'{len(df_i)} numeros inséré dans la base de données')
    if not df_e.empty:
        print(f'{len(df_e)} numeros déjà existant dans la base de données')
elif not df_e.empty:
    print(f'{len(df_e)} numeros déjà existant dans la base de données')
else:
    print('Aucune insertion dans la base de données')
con.commit()
con.close()

In [11]:
def row_num(uploaded_file, word_contains):
    wb = px.load_workbook(uploaded_file)
    ws = wb.worksheets[0]
    for row in ws.iter_rows():
        for cell in row:
            if word_contains in str(cell.value):
                num_row = cell.row
                break
        if num_row:
            break
    return num_row

In [12]:
def col_num(uploaded_file, word_contains,num_row):
    wb = px.load_workbook(uploaded_file)
    ws = wb.worksheets[0]
    if num_row is not None:
        for cell in ws[num_row]:
            if cell.value is not None and cell.value == word_contains:
                num_col = cell.column
    return num_col

In [13]:
def process_excel_file(uploaded_file,min_row,max_row,min_col,max_col):
    wb = px.load_workbook(uploaded_file) # Load the workbook
    ws = wb.worksheets[0] # Load worksheet
    for row in ws.iter_rows():
        for cell in row:
            if cell.value:  # Check if the cell is not empty
                cell.value = unidecode(str(cell.value)).upper()
    data = []
    for row in ws.iter_rows(min_row=min_row, max_row=max_row, min_col=min_col, max_col=max_col):
        data.append([cell.value for cell in row])
    df = pd.DataFrame(data) # Create a dataframe from the data
    df.columns = df.iloc[0] # Consider second line of dataframe as column names
    return df    

In [14]:
file=r"C:\Users\nmaxi\OneDrive\Bureau\AffiKoue\test.xlsx"
min_row=row_num(file,"PRENOMS")
max_row=row_num(file,"NB:")
min_col=1
max_col=col_num(file,"NOM CONFORME")
df=process_excel_file(wb,min_row,max_row,min_col,max_col)

UnboundLocalError: cannot access local variable 'num_row' where it is not associated with a value

In [ ]:
# Function to process the Excel file and insert data into the database
def process_excel_file(uploaded_file):
    """_summary_ : load file from excel file and transform into dataframe

    Args:
        uploaded_file (_type_): _description_ : excel file 

    Returns:
        _type_: _description_ : dataframe of phone number and name of owner
    """
    # Load the workbook
    wb = px.load_workbook(uploaded_file)
    ws = wb.worksheets[0]

    # Initialize variables
    min_row = None
    max_row = None
    min_col = 1
    max_col = None

    # Find the min_row containing "NOM ET PRENOM"
    for row in ws.iter_rows(min_row=1, max_col=ws.max_column):
        for cell in row:
            if "NOM ET PRENOM" in str(cell.value):
                min_row = cell.row
                break
        if min_row:
            break

    # Find the max_row in the first column that contains a digit
    for row in ws.iter_rows(min_row=min_row, max_row=ws.max_row, min_col=1, max_col=1):
        for cell in row:
            if isinstance(cell.value, int):
                max_row = cell.row

    # Find the last column of the min_row containing "NOM CONFORME"
    if min_row is not None:
        for cell in ws[min_row]:
            if cell.value is not None and cell.value == "NOM CONFORME":
                max_col = cell.column

    # Specify the range
    rows = ws.iter_rows(min_row=min_row, max_row=max_row, min_col=min_col, max_col=max_col)

    data = []
    for row in rows:
        data.append([cell.value for cell in row])

    # Create a dataframe from the data
    df = pd.DataFrame(data)
    # Consider second line of dataframe as column names
    df.columns = df.iloc[0]
    # delete column whose name doesn't contains list of strings
    df = df[df.filter(regex="Nom|NOM|nom|Prénom|PRENOM|prenom|prénom|phone|Phone|PHONE|STATUT").columns]

    # Drop 1st row
    df = df.drop(0)

    # Keep only last 4 columns
    df = df.iloc[:, -4:]

    # Rename first column to 'numero'
    df.rename(columns={df.columns[0]: 'numero'}, inplace=True)

    # subset the dataframe to keep only rows where statut is not active and NOM conforme is not 1 or len of phone number is not 8, 10, 11 or 13
    #df_nc = df[(df['STATUT'] != 'ACTIVE') | (df['NOM CONFORME'] != 1) | (df['numero'].apply(lambda x: len(str(x)) not in [8, 10, 11, 13]))]

    # Add "229" to numbers where lengths is 10 digits and start with 01, "22901" where lengths is 8 digits,
    # insert 01 between 229 and the rest of the number where lengths is 11
    # keep as is it where len is 13 digits and raise errors for other cases
    df['numero'] = df['numero'].apply(lambda x: '229' + x if len(str(x)) == 10 and str(x).startswith('01')
                                      else ('22901' + x if len(str(x)) == 8 else ('22901' + str(x)[3:] if len(str(x)) == 11 else 'error')))

    # Convert only columns whose contains phone, or Phone or PHONE to numeric
    df['numero'] = df['numero'].apply(pd.to_numeric, errors='coerce')

    # Drop rows with NaN values for phone numbers
    df = df.dropna(subset=['numero'])

    # subset the dataframe to keep only rows where statut is active and NOM conforme is 1
    df_c = df[(df['STATUT'] == 'ACTIVE') & (df['NOM CONFORME'] == 1)]

    # rename 2nd column in nom
    df_c.rename(columns={df.columns[1]: 'nom'}, inplace=True)

    # Drop columns whose not contains nom or numero
    df_c = df_c[df_c.filter(regex="nom|numero").columns]
    
    return df_c


# Function to insert data into sql_table
def register_file(df):
    """_summary_ : Function to insert rows into sqlite table

    Args:
        df (_type_): _description_ : dataframe that contain rows to insert

    Returns:
        _type_: _description_ : dataframe of rows inserted and dataframe of rows
        already in the dataframe
    """
    # Create a connection to a sqlite database
    con = sql.connect('data/AffiKoue.db')
    # Create a cursor
    c = con.cursor()
    # fetching number values from sqlite table momo
    num_exist = pd.read_sql_query("SELECT numero FROM momo", con)['numero'].tolist()
    # create dataframe for only rows where numero doesn't exist in table data
    df_i = df[~df['numero'].isin(num_exist)]
    # create dataframe for only rows where numero exist in table data
    df_e = df[df['numero'].isin(num_exist)]
    # insert df_i rows into table momo in dataframe
    if not df_i.empty:
        df_i.to_sql('momo', con, if_exists='append', index=False)
        st.success(f'{len(df_i)} numeros inséré dans la base de données')
        if not df_e.empty:
            st.warning(f'{len(df_e)} numeros déjà existant dans la base de données')
    elif not df_e.empty:
        st.warning(f'{len(df_e)} numeros déjà existant dans la base de données')
    else:
        st.info('Aucune insertion dans la base de données')
    con.commit()
    con.close()

    return df_i, df_e


# Function to find number 
def find_number(df):
    """_summary_ : function to find find number already registered in the sqlite table

    Args:
        df (_type_): _description_ : dataframe that contain rows to verify if 
        already exist in the dataframe

    Returns:
        _type_: _description_ : dataframe of rows present in the dataframe and rows doesn't present in the dataframe
    """
    # Create a connection to a sqlite database
    con = sql.connect('data/AffiKoue.db')
    # Create a cursor
    c = con.cursor()
    # fetching number values from sqlite table momo
    num_exist = pd.read_sql_query("SELECT numero FROM momo", con)['numero'].tolist()
    con.commit()
    con.close()
    # create dataframe for only rows where numero exist in table data
    df_e = df[df['numero'].isin(num_exist)]
    # create dataframe for numero doesn't exist in table data
    df_i = df[~df['numero'].isin(num_exist)]
    # insert df_i rows into table momo in dataframe
    if not df_e.empty:
        df_e.rename(columns={'numero':'NUMERO DE TELEPHONE',
                     'nom':'NOM ET PRENOMS MOMO'},inplace=True)
        df_e['STATUT']="Vérifié"
    elif not df_i.empty:
        st.warning(f"{len(df_i)} numeros n'ont pas été retrouvé")
        df_i['STATUT']="Non retrouvé" 
        
    return df_e, df_i


def df_to_excel(df,start_row,hsize,csize,num_col1: list,col_name: list, col_width: list):
    """_summary_ : function to export data from momo dataframe to excel

    Args:
        df (_type_): _description_ : dataframe to export
        starting row of table
        num_col: numeric column to format
    """
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False,sheet_name='LISTE MOMO',startrow=start_row)
        workbook = writer.book
        ws = writer.sheets['LISTE MOMO']
        # Define custom styles
        header_font = Font(bold=True, color="100101",size=hsize)
        header_fill = PatternFill(start_color="EFFA4F", end_color="EFFA4F", fill_type="solid")
        cell_font= Font(color="100101",size=csize)

        # Define border style 
        top=Side(border_style="thin",color="100101")
        bottom=Side(border_style="thin",color="100101")
        left=Side(border_style="thin",color="100101")
        right=Side(border_style="thin",color="100101")
        
        border=Border(top=top,bottom=bottom,left=left,right=right)
        
        # Apply styles to header
        for cell in ws[start_row+1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.border=border

        # Apply styles for values
        for row in ws.iter_rows(min_row=start_row+1, max_row=start_row+len(df)+1, min_col=1, max_col=df.shape[1]):
            for cell in row:
                cell.border=border
                cell.font=cell_font
                if cell.column - 1 in num_col1:  # Adjusting for 0-based index
                    cell.number_format = numbers.FORMAT_NUMBER
        
         # Set column widths
        for col, width in zip(col_name, col_width):
            ws.column_dimensions[col].width = width
        
        # Set row heights
        for row in range(start_row, start_row+len(df)+2):
            ws.row_dimensions[row].height = 30  # Set height for all rows
            
        # Ensure formulas are recognized as formulas
        for row in ws.iter_rows(min_row=start_row + 1, max_row=start_row + len(df) + 1, min_col=1, max_col=df.shape[1]):
            for cell in row:
                if isinstance(cell.value, str) and cell.value.startswith('='):
                    cell.value = f"={cell.value[1:]}"
    
    return output

def add_text_to_sheet(output,df,text: list,name : list,label_:list,def_value: list,
                      row:list,col:list,before_after: list):
    """_summary_ : function to add text to existing file 

    Args:
        output (_type_): bytes0 object to get excel file from
        df : dataframe data already contain in the excel file
        text : list of each text to add on the excel sheet
        name : list of each object to store this text in the function 
        label : list of A short label explaining to the user what this input is for
        def_value:The text value of this widget when it first renders. 
        This will be cast to str internally
        row: the row position of text in sheet
        col : the col index position in sheet
        before_after: list to precise each time if the rows is before of after
        existing dataframe data. if it's before just use row index provide if it's after 
        use row index + len(df)+1
        
    """
    # Load the workbook from the BytesIO object
    output.seek(0)
    wb = px.load_workbook(output)
    ws = wb.active

    # Add text to specified cells
    for i, txt in enumerate(text):
        if before_after[i] == 'before':
            ws.cell(row=row[i], column=col[i], value=txt)
        elif before_after[i] == 'after':
            ws.cell(row=row[i] + len(df) + 1, column=col[i], value=txt)

    # Save the modified workbook back to the BytesIO object
    new_output = BytesIO()
    wb.save(new_output)
    new_output.seek(0)
    return new_output

def download_button(output,label,file_name):
    """_Summary_: function to download file 
    Args:
        Output: File to download
        label : label of button
        file_name : file name to should finish by .xlsx
    """
    st.markdown("""
    <style>
    div.stButton {text-align:center}
    </style>""", unsafe_allow_html=True)    
    st.download_button(
        label=label,
        data=output.getvalue(),
        file_name=file_name,
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )

def df_to_demande_payement(df,start_row):
    df=df[df.filter(regex="PHONE|MOMO").columns]
    df.rename(columns={'NOM ET PRENOMS MOMO':'NOM ET PRENOMS'},inplace=True)
    
    num_ordre = []
    for i in range(len(df)):
        if i == 0:
            num_ordre.append(1)
        else:
            num_ordre.append(f"=A{i+start_row+1}+1")
    df["NUM D'ORDRE"] = num_ordre
    
    # Add FRAIS column
    frais = []
    for i in range(len(df)):
        frais.append(f"""=IFS(isblank(D{start_row + i + 2}),"",D{start_row + i + 2}>=500001, 5000,
                     D{start_row + i + 2}>=300001,3500, D{start_row + i + 2}>=200001, 3000, 
                     D{start_row + i + 2}>=100001, 2000, D{start_row + i + 2}>=50001, 1000, 
                     D{start_row + i + 2}>=20001, 700, D{start_row + i + 2}>=10001, 350, 
                     D{start_row + i + 2}>=5001, 200,  D{start_row + i + 2}>=501, 100, 
                     D{start_row + i + 2}>=1, 50)""")
    df["FRAIS"] = frais
    
    # Add TOTAL column
    total = []
    for i in range(len(df)):
        total.append(f"=D{start_row + i + 2} + E{start_row + i + 2}")
    df["TOTAL"] = total
    
    df["MONTANT"]=np.nan
    
    df_momo=df
    
    df_momo=df_momo[["NUM D'ORDRE",'NOM ET PRENOMS','NUMERO DE TELEPHONE',"MONTANT","FRAIS","TOTAL"]]
    return df_momo

# Streamlit app
st.markdown("<h1 style='text-align: center; white: red;'>AffiKoue</h1>", unsafe_allow_html=True)

# Custom CSS for specific elements
st.markdown(
    """
    <style>
    .stApp {
        background-color: #205374;  /* light blue background */
    }
    h1 {
        color: #FF5733;  /* Orange header */
    }
    .sidebar .sidebar-content {
        background-color: #1c2833;  /* Dark sidebar */
        color: #FFFFFF;  /* White text in sidebar */
    }
    .stAlert {
        background-color: #566573;  /* grey background for alerts */
        color: red;  /* red text for alerts */
    }
    .stSuccess {
        background-color: #566573;  /* grey background for success messages */
        color: #155724;  /* Dark green text for success messages */
    }
    .stWarning {
        background-color: #566573;  /* grey background for success messages */
        color: #f4d03f;  /* Dark yellow text for warnings */
    }
    .stInfo {
        background-color: #566573;  /* grey background for success messages */
        color: #2471a3;  /* Dark blue text for info messages */
    }
    .stFileUploader {
        font-size: 14px;  /* Smaller font size */
        padding: 5px 10px;  /* Smaller padding */
    }
    .stFileUploader label {
        color: #FF5733;  /* Change the color of the file uploader label */
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Sidebar options
st.sidebar.title("Options")
option = st.sidebar.selectbox("MOMO", ["Chercher un numero","Enregistrer numero"],index=None)

if option=="Enregistrer numero":
    uploaded_file = st.file_uploader("Choisir un fichier au format adéquat", type=["xlsx"])

    if uploaded_file is not None:
        df_c=process_excel_file(uploaded_file)
        df_i, df_e= register_file(df_c)
        
elif option=="Chercher un numero":
    uploaded_file = st.file_uploader("Choisir un fichier au format adéquat", type=["xlsx"])
    if uploaded_file is not None:
        df_c=process_excel_file(uploaded_file)
        df_e,df_i= find_number(df_c)
        if not df_e.empty:
            output=df_to_excel(df_e,start_row=2,hsize=16,csize=14,num_col1=[0],
                               col_name=["A","B","C"],col_width=[30,50,20])
            download_button(output,"Telecharger les numeros enrégistrés","numeros_enregsitres.xlsx")
        if not df_i.empty:
            output=df_to_excel(df_i,start_row=2,hsize=16,csize=14,num_col1=[0],
                               col_name=["A","B","C"],col_width=[30,50,20])            
            download_button(output,"Telecharger les numeros non retrouvés","numeros_non_retrouves.xlsx")
        if not df_e.empty:
            df_momo=df_to_demande_payement(df_e,7)
            output=df_to_excel(df_momo,start_row=7,hsize=16,csize=14,num_col1=[2],
                               col_name=["A","B","C","D","E","F"],col_width=[15,50,30,30,20,30])
            download_button(output,"Générer état de payement","demande_momo.xlsx")

In [ ]:



def saisie_activite(objet):
    req_objet=objet
    # Connexion à la base de données
    con = sql.connect("data/finance.db")
    c = con.cursor()
    # Saisie des données
    if objet=="Reconciliation":
        with st.form("reconciliation_form"):
            req_code=st.selectbox("Code la requête : ", valeurs_uniques("data/finance.db", "requete", "req_code"),index=None)
            req_reconciliation=st.number_input("Montant de la réconciliation : ",step=1,value=None)
            if req_code:
                query = '''
                SELECT req_code,b_code_activite,req_nom,req_type,req_montant,b_projet,b_code_resultat,b_item_code,req_demandeur,req_departement
                FROM requete
                WHERE req_code = ?
                '''
                c.execute(query, (req_code,))
                # Récupération du projet, code résultats et item code de la table budget
                req_code,b_code_activite,req_nom,req_type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement = c.fetchone()
            submitted = st.form_submit_button("Ajouter")
            if submitted:
                c.execute(" INSERT INTO requete(req_reconciliation,req_objet,req_code,b_code_activite,req_nom,req_type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)",(req_reconciliation,req_objet, req_code,b_code_activite,req_nom,req_type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement))
                # Valider et fermer la connexion
                con.commit()
                con.close()
                st.success(f"L'activité {req_nom} avec une dépense finale de {req_reconciliation}FCFA a été réconcilié.")
    elif objet=="Requête à annuler":
        with st.form("annulation_form"):
            req_code=st.selectbox("Code la requête : ", valeurs_uniques("data/finance.db", "requete", "req_id"),index=None)
            if req_code:
                query = '''
                SELECT req_code,b_code_activite,req_nom,req_
                type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement
                FROM requete
                WHERE req_id = ?
                '''
                c.execute(query, (req_code,))
                # Récupération du projet, code résultats et item code de la table budget
                req_code,b_code_activite,req_nom,req_type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement = c.fetchone()
                req_montant=req_montant*-1
            submitted = st.form_submit_button("Ajouter")
            if submitted:
                c.execute(" INSERT INTO requete(req_objet,req_code,b_code_activite,req_nom,req_type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement) VALUES (?,?,?,?,?,?,?,?,?,?,?)",(req_objet, req_code,b_code_activite,req_nom,req_type,req_montant, b_projet, b_code_resultat, b_item_code,req_demandeur,req_departement))
                # Valider et fermer la connexion
                con.commit()
                con.close()
                st.success(f"L'activité {req_nom} a été annulé avec succès.")
    else:
        with st.form("form_activite"):
            req_nom = st.text_input("Nom de l'activité : ")
            req_type= st.selectbox("Type de requête", ["Requete de voyage", "Achat de biens ou services"],index=None)
            req_montant = st.number_input("Montant de l'activité : ",step=1,value=None)
            req_demandeur=st.selectbox("Demandeur : ", valeurs_uniques("data/finance.db", "users", "nom_prenom"),index=None)
            b_code_activite = st.selectbox("Code activité", valeurs_uniques("data/finance.db", "budget", "b_code_activite"),index=None)
            if b_code_activite:
                # Query pour récupérer le projet, le code résultat et l'item code pour l'activité donnée
                query = '''SELECT b_projet, b_code_resultat, b_item_code FROM budget WHERE b_code_activite = ?'''
                c.execute(query, (b_code_activite,))
                # Récupération du projet, code résultats et item code de la table budget
                b_projet, b_code_resultat, b_item_code = c.fetchone()
            if  objet=="Requête complémentaire":
                req_code=st.selectbox("Code de la requête : ", valeurs_uniques("data/finance.db", "requete", "req_code"),index=None) 
            submitted = st.form_submit_button("Ajouter")
            if submitted:
                if objet=="Requête complémentaire":
                    c.execute("INSERT INTO requete (req_code,req_objet,req_nom,req_type,req_montant, req_demandeur, b_code_activite, b_projet , b_code_resultat, b_item_code) VALUES (?,?,?,?,?,?,?,?,?,?)", (req_code,req_objet,req_nom,req_type,req_montant, req_demandeur, b_code_activite, b_projet , b_code_resultat, b_item_code))
                if objet=="Requête initiale":
                    c.execute("INSERT INTO requete (req_objet,req_nom,req_type,req_montant, req_demandeur, b_code_activite, b_projet , b_code_resultat, b_item_code) VALUES (?,?,?,?,?,?,?,?,?)", (req_objet, req_nom,req_type,req_montant, req_demandeur, b_code_activite, b_projet , b_code_resultat, b_item_code))
                    # Récupérer l'ID généré par autoincrement
                    req_code = c.lastrowid
                    # Mettre à jour la ligne qui vient d'être insérér en ajoutant le code de la requête sur cette ligne
                    c.execute(f""" UPDATE requete SET req_code= ? WHERE req_id = ?""", (req_code,req_code))
                # recuperer le département de l'utilisateur
                query = '''SELECT departement FROM users WHERE nom_prenom = ?'''
                c.execute(query, (req_demandeur,))
                req_departement = c.fetchone()[0]
                # Mettre à jour la ligne qui vient d'être insérér en ajoutant le code de la requête sur cette ligne
                c.execute(f""" UPDATE requete SET req_departement= ? WHERE req_id = ?""", (req_departement,c.lastrowid))
                # Valider et fermer la connexion
                con.commit()
                con.close()
                    # Afficher l'ID généré
                st.success(f"L'activité a été ajoutée avec succès. Code de la requête : {req_code}")

In [ ]:

def afficher_solde():
    # Connexion à la base de données
    con = sql.connect("data/finance.db")
    c = con.cursor()
    #Recuperation des données de la table solde
    df_solde = pd.read_sql("SELECT * FROM budget", con)
    # Fitrer le solde par code_activite
    b_code_activite = st.selectbox("Code activité : ", valeurs_uniques("data/finance.db", "budget", "b_code_activite"),index=None)
    df_solde = df_solde[df_solde["b_code_activite"] == b_code_activite]
    #Affichage des données
    st.dataframe(df_solde)
    # Valider et fermer la connexion
    con.commit()
    con.close()

In [ ]:
    
def user_register(db_path,form_name):
    con = sql.connect(db_path)
    c= con.cursor()
    # Saisie des données
    with st.form(form_name):
        nom_prenom = st.text_input("Nom et prénom : ")
        departement=st.selectbox("departement", ["DRHA", "DFA", "DHASE", "DRSE", "DSR/SMNI", "DMMC", "DAI", "DARS3"],index=None)
        submitted = st.form_submit_button("Ajouter")
        if submitted:
            df=pd.read_sql("SELECT * FROM users", con)
            if df['nom_prenom'].str.contains(nom_prenom).any():
                st.error(f"Utilisateur {nom_prenom} déjà dans la base")
            else:
                c.execute("INSERT INTO users (nom_prenom, departement) VALUES (?, ?)", (nom_prenom, departement))
                # Valider et fermer la connexion
                con.commit()
                con.close()
                st.success(f"Utilisateur {nom_prenom} bien ajouté") 

In [ ]:
"""elif user_option=="Liste des utilisateurs":
        st.selectbox("Liste des utilisateurs",valeurs_uniques("data/finance.db","users","nom_prenom"),index=None)  
    elif budget_option == "Ajouter budget":
        fichier_xlsx = st.file_uploader("Télécharger le fichier Excel", type=["xlsx"])
        if fichier_xlsx:
            inserer_budget(fichier_xlsx)
    
    elif requete_option=="Requête initiale":
        saisie_activite(objet=requete_option)
        calculer_solde()
    
    elif requete_option=="Requête complémentaire":
        saisie_activite(objet=requete_option)
        calculer_solde()
        
    elif requete_option=="Requête à annuler":
        saisie_activite(objet=requete_option)
        calculer_solde()
    elif requete_option=="Reconciliation":
        saisie_activite(objet=requete_option)
        calculer_solde()
    elif dashboard_option == "Afficher solde":
        afficher_solde()    
"""

In [3]:
con=sql.connect("data/finance.db")


In [5]:
con.close() # fermer la connection

In [8]:
def creer_table(db_path, name, pkey, cols: list, cols_type=[], pkey_type=None, autoincrement=None):
    con = sql.connect(db_path)
    c = con.cursor()
    c.execute(f"PRAGMA table_info({name})")
    if c.fetchone() is None:
        # Create the table with the primary key and additional columns
        columns = ", ".join([f"{col} {col_type}" for col, col_type in zip(cols, cols_type)]) if cols_type else ", ".join(cols)
        pkey_definition = f"{pkey} {pkey_type} PRIMARY KEY" if pkey_type else f"{pkey} PRIMARY KEY"
        if autoincrement:
            pkey_definition += f" {autoincrement}"
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {name} ({pkey_definition}, {columns})"""
        c.execute(create_table_query)
        if autoincrement is not None:
            c.execute(f"""UPDATE sqlite_sequence SET seq=999 WHERE name='{name}'""")
    con.commit()  # enregistrer
    con.close()  # fermer la connection

In [9]:
db_path="data/finance1.db"

# Créer la table de la requete
cols=["req_objet","req_code","b_code_activite","req_nom","req_type","req_montant","req_date","req_reconciliation","b_projet","b_code_resultat","b_item_code","req_demandeur","req_departement"]
cols_type=["TEXT","TEXT","INTEGER","TEXT","TEXT","INTEGER","DATETIME DEFAULT CURRENT_TIMESTAMP","INTEGER","TEXT","TEXT","TEXT","TEXT","TEXT"]
creer_table(db_path=db_path,name="requete",pkey="req_id",cols=cols,cols_type=cols_type,pkey_type="INTEGER",autoincrement="AUTOINCREMENT")

In [11]:
def valeurs_sql_col(db_path,table,var_key,cols:list):
    con = sql.connect(db_path)
    c = con.cursor()
    col_str = ", ".join(cols)
    query = f"SELECT {col_str} FROM {table} WHERE {var_key} = ?"
    c.execute(query, (var_key,))
    result = c.fetchone()
    con.close()
    return list(result) if result else None

In [ ]:
db_path="data/finance.db"
cols=valeurs_sql_col(db_path,"budget","b_code_activite",["b_projet","b_code_resultat","b_item_code"])
print(cols)

None


In [ ]:
def creer_table(db_path, name, pkey, cols: list, cols_type=[], pkey_type=None, autoincrement=None):
    con = sql.connect(db_path)
    c = con.cursor()
    c.execute(f"PRAGMA table_info({name})")
    if c.fetchone() is None:
        # Create the table with the primary key and additional columns
        columns = ", ".join([f"{col} {col_type}" for col, col_type in zip(cols, cols_type)]) if cols_type else ", ".join(cols)
        pkey_definition = f"{pkey} {pkey_type} PRIMARY KEY" if pkey_type else f"{pkey} PRIMARY KEY"
        if autoincrement:
            pkey_definition += f" {autoincrement}"
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {name} ({pkey_definition}, {columns})"""
        c.execute(create_table_query)
        if autoincrement is not None:
            c.execute(f"""UPDATE sqlite_sequence SET seq=999 WHERE name='{name}'""")
    con.commit()  # enregistrer
    con.close()  # fermer la connection

In [ ]:
def valider_form(form_data):
    return all(bool(value.strip()) if isinstance(value, str) else value is not None for value in form_data.values())

In [3]:
df = pd.DataFrame({"fruit" : ["a","b","c","b","a"],
                   "type" : ["good","good","good","good","bad"],
                   "price" : [10,15,20,25,30]})

In [ ]:
df['depense']=np.where(df['type']=="good")

,fruit,type,price
0,a,good,10
1,b,good,15
2,c,good,20
3,b,good,25
4,a,bad,30


In [ ]:
def calculer_solde(db_path,tab1,tab2,tab1_col1,tab1_col2,tab1_colf,tab2_col1,tab2_col2,var_key):
    con = sql.connect(db_path)
    c = con.cursor()
    # Récupérer les données des deux tables
    df1 = pd.read_sql(f"SELECT * FROM {tab1}", con)
    df2 = pd.read_sql(f"SELECT * FROM {tab2}", con)
    df2['x']=np.where(df2[f'{tab2_col1}']>=1,df2[f'{tab2_col1}'],df2[f'{tab2_col2}'])
    # Faire la somme par groupe dans la table requete et l'affecter à la table budget
    df1[tab1_col1]=df1[f'{var_key}'].map(df2.groupby(f"{var_key}")["x"].sum()) 
    # Mettre à jour la table existante
    df1.to_sql(f"{tab1}", con, if_exists='replace', index=False)
    # Requête SQL pour effectuer la soustraction directement entre les tables
    query = f""" UPDATE {tab1} SET {tab1_colf} = {tab1_col2} - {tab1_col1}"""
    c.execute(query)
    con.commit()
    con.close()

In [ ]:
def filtrer_afficher_df(df_budget):
    selected_filtre = st.selectbox("Filtrer", valeurs_uniques("data/finance.db", "solde", "b_code_activite"),index=None) # Fitrer le solde par code_activite
    filtered_df = df_budget[df_budget["code_activite"] == selected_filtre]
    #Affichage des données
    st.dataframe(filtered_df)

In [ ]:
def depense_base_requete(code_requete):
    con, c= conn() 
    df=pd.read_sql(""" SELECT * FROM requete """,con)
    df2=(df.groupby("code_requete")["r_montant"].sum()).to_frame().reset_index()
    depense=(df2.loc[df2["code_requete"]==code_requete, "r_montant"].values)[0].item()
    # mettre à jour la dépense dans la table requete où le type d'activité est 'Requête initiale'
    c.execute("UPDATE requete SET r_depense=? WHERE code_requete=? AND type_activite='Requête initiale'", (depense, code_requete))
    con.commit()
    con.close()

In [7]:
con = sql.connect("data/finance.db")
c = con.cursor()
#df=pd.read_sql("SELECT * FROM budget", con)


In [1]:
def depense_par_code_activite():
    con = sql.connect("data/finance.db")
    df_budget=pd.read_sql(""" SELECT * FROM budget """,con)
    df_requete=pd.read_sql(""" SELECT * FROM requete """,con)
    df_budget["b_depense"]=df_budget["b_code_activite"].map(df_requete.groupby("r_code_activite")["r_depense"].sum())
    df_budget.to_sql("budget", con,if_exists="replace", index=False)

In [6]:
def col_budget_to_df(columns_old_names, new_names):
    con = sql.connect("data/finance.db")
    df = pd.read_sql("SELECT * FROM budget", con) #Recuperation des données de la table budget
    if len(columns_old_names) != len(new_names):
        raise ValueError("Le nombre de colonnes sélectionnées doit correspondre au nombre de nouveaux noms.")
    df_budget = df[columns_old_names].copy() # Sélectionner les colonnes et les renommer
    df_budget.columns = new_names
    con.close()
    return df_budget

In [11]:
def recuperer_valeurs_colonne(table_name,var_key,cols:list):
    con = sql.connect("data/finance.db")
    c=con.cursor()
    col_str = ", ".join(cols)
    query = f"""SELECT {col_str} FROM {table_name} WHERE {var_key} = ?"""
    c.execute(query, (var_key,))
    result = c.fetchone()
    con.close()
    return list(result) if result else None

In [15]:
def depense_par_code_activite():
    con = sql.connect("data/finance.db")
    df_budget=pd.read_sql(""" SELECT * FROM budget """,con)
    df_requete=pd.read_sql(""" SELECT * FROM requete """,con)
    df_reconciliation=df_requete[df_requete["type_activite"]=="Reconciliation"]
    df_requete_emis=df_requete[df_requete["type_activite"]!="Reconciliation"]
    df_req_non_reconcilie=df_requete_emis[~df_requete_emis["code_requete"].isin(df_reconciliation["code_requete"])]
    df_depenses=pd.concat([df_req_non_reconcilie,df_reconciliation],axis=0)
    df_budget["b_depense"]=df_budget["b_code_activite"].map(df_depenses.groupby("r_code_activite")["r_montant"].sum())
    df_budget["b_depense"]=df_budget["b_depense"].fillna(0).astype(int)
    return df_budget, df_depenses, df_reconciliation, df_req_non_reconcilie, df_requete_emis, df_requete

In [ ]:
user_option=st.sidebar.selectbox("Utilisateurs", ["Ajouter utilisateurs", "Liste des utilisateurs"],index=None)   

In [ ]:
# Actions 
    if user_option=="Ajouter utilisateurs":
        with st.form("Utilisateurs"):
            nom_prenom=st.text_input("Nom de l'utilisateur: ")
            u_departement=st.selectbox("Departement : ",["DFA","DRHA","DMMC", "DSR/SMNI", "DRSE", "DAI", "DARS3","DHASE"],index=None)
            if st.form_submit_button("Ajouter"):
                find=verifier_utilisateurs_existant(nom_prenom)
                if find:
                    st.error(f" Utilisateur {nom_prenom} déjà enrégistré")
                else :
                    inserer_utilisateurs(nom_prenom, u_departement)
                    st.success(f" Utilisateur {nom_prenom} a été bien enrégistré")
    elif user_option=="Liste des utilisateurs":
        con,c=conn()
        df=pd.read_sql("SELECT * FROM users", con)
        df.columns=["Utilisateurs", "Departement"]
        filtre=st.selectbox("Utilisateurs",valeurs_uniques("users","nom_prenom"),index=None)
        st.dataframe(df[df["Utilisateurs"]==filtre],hide_index=True)

In [16]:
df_budget, df_depenses, df_reconciliation, df_req_non_reconcilie, df_requete_emis, df_requete = depense_par_code_activite()

In [ ]:
st.write("Consommation par Code Activité")
            r_code_activite = st.selectbox("Code Activité", valeurs_uniques("budget", "b_code_activite"), index=None)
            if r_code_activite:
                df = col_budget_to_df(["b_code_activite", "b_depense", "b_montant"], ["Code activité", "Depense", "Montant"])
                df["Conssomation"] = df["Depense"] / df["Montant"]
                df_consommation = df[df["Code activité"] == r_code_activite]
                df_consommation["Code activité"] = df_consommation["Code activité"].astype(str)
                # Mettre la consommation en pourcentage
                df_consommation["Consommation"] = df_consommation["Conssomation"] * 100
                st.bar_chart(df_consommation.set_index("Code activité")["Consommation"])       

In [17]:
df_reconciliation

,id,id_requete,type_activite,nom,code_requete,type_requete,demandeur,r_code_activite,r_montant,date,r_projet,r_code_resultat,r_item_code,r_departement


In [57]:
con = sql.connect("data/finance.db")
c = con.cursor()
df=pd.read_sql(""" SELECT * FROM requete """,con)
df2=(df.groupby("code_requete")["r_montant"].sum()).to_frame().reset_index()
    

In [38]:
df2

,code_requete,r_montant
0,1000,15000
1,1001,150
2,1002,1500
3,1003,15008
4,1004,1500


In [55]:
con.close()

In [58]:
code_requete=1001
depense=(df2.loc[df2["code_requete"]==code_requete, "r_montant"].values)[0].item()
c.execute("UPDATE requete SET r_depense=? WHERE code_requete=? AND type_activite='Requête initiale'", (depense, code_requete))
con.commit()
con.close()

In [ ]:
con = sql.connect("data/finance.db")
c = con.cursor()
df_budget=pd.read_sql(""" SELECT * FROM budget """,con)
df_requete=pd.read_sql(""" SELECT * FROM requete """,con)
df_budget["b_depense"]=df_budget["b_code_activite"].map(df_requete.groupby("r_code_activite")["r_depense"].sum())

In [68]:
df_budget

,b_code_activite,b_projet,b_code_resultat,b_item_code,b_montant,b_depense,solde,b_departement
0,1021,P24,None,A7100,15000000,0.0,15000000,DHASE
1,1022,P36,None,A6100,15000000,NaN,None,DRSE
2,1023,P36,None,A6100,15000000,NaN,None,DRSE
3,1024,P36,None,A6100,15000000,NaN,None,DRSE
4,1025,P36,None,A6200,15000000,NaN,None,DRSE
5,1026,P24,None,A7200,17000000,150.0,17000000,DHASE
6,1027,P36,None,A6200,17000000,15200.0,17000000,DAI
7,1028,P24,None,A7100,17000000,NaN,None,DHASE
8,1029,P24,None,A7200,17000000,NaN,None,DHASE
9,1030,P36,None,A6200,6000000,NaN,None,DAI


In [71]:
df_requete

,id,type_activite,nom,code_requete,type_requete,demandeur,r_code_activite,r_montant,reconciliation,r_depense,date,r_projet,r_code_resultat,r_item_code,r_departement
0,1000,Requête initiale,MISSION 1,1000,Avance de voyage,ABLET Juliana,1021,15000,None,NaN,2025-02-23 11:24:03,P24,None,A7100,DRHA
1,1001,Requête initiale,MISSION 1,1001,Avance de voyage,ABLET JULIANA,1026,150,None,150.0,2025-02-23 11:42:41,P24,None,A7100,DMMC
2,1002,Requête initiale,MISSION 1,1002,Avance de voyage,ABLET Juliana,1027,1500,None,NaN,2025-02-23 11:56:52,P24,None,A7100,DRHA
3,1003,Requête initiale,MISSION 1,1003,Avance de voyage,ABLET JULIANA,1027,15008,None,NaN,2025-02-23 12:01:28,P24,None,A7100,DMMC
4,1005,Requête initiale,MAXIES,1005,Achat de biens ou de service,ABLET JULIANA,1027,15200,None,15200.0,2025-02-23 19:59:31,P24,None,A7100,DMMC


In [65]:
df_requete.groupby("r_code_activite")["r_depense"].sum()

r_code_activite
1021        0.0
1026      150.0
1027    15200.0
Name: r_depense, dtype: float64